In [1]:
import ee
import os
import geojson
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
from geemap import png_to_gif
from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
ee.Initialize()
%matplotlib inline

In [27]:
filename = "gibraltar.geojson"
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

In [28]:
start_date = "2021-12-01" # 
end_date = "2021-12-31" #

In [29]:
ffa_db = (ee.ImageCollection("COPERNICUS/S2_SR")
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))

In [41]:
ffa_db.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'COPERNICUS/S2_SR',
 'version': 1643878706244514,
 'properties': {'date_range': [1490659200000, 1643673600000],
  'period': 0,
  'system:visualization_0_min': '0.0',
  'type_name': 'ImageCollection',
  'keywords': ['copernicus',
   'esa',
   'eu',
   'msi',
   'reflectance',
   'sentinel',
   'sr'],
  'system:visualization_0_bands': 'B4,B3,B2',
  'thumb': 'https://mw1.google.com/ges/dd/images/COPERNICUS_S2_SR_thumb.png',
  'description': '<p>Sentinel-2 is a wide-swath, high-resolution, multi-spectral\nimaging mission supporting Copernicus Land Monitoring studies,\nincluding the monitoring of vegetation, soil and water cover,\nas well as observation of inland waterways and coastal areas.</p><p>The Sentinel-2 L2 data are downloaded from scihub. They were\ncomputed by running sen2cor. WARNING: ESA did not produce L2 data\nfor all L1 assets, and earlier L2 coverage is not global.</p><p>The assets contain\n12 UINT16 spectral bands representin

In [30]:
import time
acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

['12/02/21',
 '12/02/21',
 '12/02/21',
 '12/02/21',
 '12/04/21',
 '12/04/21',
 '12/04/21',
 '12/04/21',
 '12/07/21',
 '12/07/21',
 '12/07/21',
 '12/07/21',
 '12/09/21',
 '12/09/21',
 '12/09/21',
 '12/09/21',
 '12/12/21',
 '12/12/21',
 '12/12/21',
 '12/12/21',
 '12/14/21',
 '12/14/21',
 '12/14/21',
 '12/14/21',
 '12/17/21',
 '12/17/21',
 '12/17/21',
 '12/17/21',
 '12/19/21',
 '12/19/21',
 '12/19/21',
 '12/19/21',
 '12/22/21',
 '12/22/21',
 '12/22/21',
 '12/22/21',
 '12/24/21',
 '12/24/21',
 '12/24/21',
 '12/24/21',
 '12/27/21',
 '12/27/21',
 '12/27/21',
 '12/27/21',
 '12/29/21',
 '12/29/21',
 '12/29/21',
 '12/29/21']

In [49]:
im_list = ffa_db.toList(ffa_db.size())
index = 10

url = ee.Image(im_list.get(index)).select("B4", "B3", "B2").clip(aoi).resample("bicubic").getThumbURL()
disp.Image(url=url, width=1200)



In [52]:
ee.Image(im_list.get(index)).clip(aoi).getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [453, 333],
   'origin': [909, 155],
   'crs': 'EPSG:32630',
   'crs_transform': [60, 0, 199980, 0, -60, 4000020]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2716, 1995],
   'origin': [5454, 930],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 199980, 0, -10, 4000020]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2716, 1995],
   'origin': [5454, 930],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 199980, 0, -10, 4000020]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [2716, 1995],
   'origin': [5454, 930],
   'crs': 'EPSG:32630',
   'crs_transform': [10, 0, 199980, 0, -10, 400

In [50]:
import requests

img_data = requests.get(url).content
with open('Images/buena.png', 'wb') as handler:
    handler.write(img_data)